In [ ]:
import math
import tensorflow
from tensorflow.compat.v1.keras.backend import get_session
tensorflow.compat.v1.disable_v2_behavior()
import kerasAC 
from kerasAC.generators.snp_generator import *
from kerasAC.tiledb_config import *
from scipy.special import softmax
from kerasAC.interpret.deepshap import * 
from kerasAC.interpret.profile_shap import * 
from kerasAC.vis import * 
from kerasAC.helpers.transform_bpnet_io import * 

In [ ]:
chd_pcgc=pd.read_csv('chd_final_0base.csv',sep='\t')
controls=pd.read_csv('chd_control_final_0base.csv',sep='\t')

print(controls.shape)
print(chd_pcgc.shape)
#controls.head()

In [ ]:
dict_var_Score_cases={}
dict_var_Score_controls={}

In [ ]:
import glob
import re
#from keras.models import load_model, model_from_json
#from keras.utils.generic_utils import get_custom_objects
from tensorflow.keras.models import load_model, model_from_json
from tensorflow.keras.utils import get_custom_objects
from kerasAC.metrics import * 
from kerasAC.custom_losses import * 
from kerasAC.metrics import * 
from kerasAC.custom_losses import * 
#from keras_genomics.layers.convolutional import RevCompConv1D
import tensorflow_probability as tfp
peak_files=glob.glob('mutations/chd_cases_*_ext.bed')
print(peak_files)
print(len(peak_files))

In [ ]:
for zz in peak_files:
    name=re.sub('_$','',zz.split('ext')[0].split('chd_cases_')[1].replace('all_','all'))
    print(name)
    case_snps_in_peaks=pd.read_csv('mutations/chd_cases_'+str(name)+'_ext.bed',sep='\t',header=None)
    case_snps_in_peaks.columns=['Chrom','Pos0','Pos']
    case_snps_in_peaks.shape
    control_snps_in_peaks=pd.read_csv('mutations/chd_controls_'+str(name)+'_ext.bed',sep='\t',header=None)
    control_snps_in_peaks.columns=['Chrom','Pos0','Pos']
    snps_inpeaks=chd_pcgc.merge(case_snps_in_peaks,on=['Chrom','Pos0','Pos'])
    snps_inpeaks=snps_inpeaks.drop_duplicates()
    print(snps_inpeaks.shape)
    control_snps_inpeaks=controls.merge(control_snps_in_peaks,on=['Chrom','Pos0','Pos'])
    control_snps_inpeaks=control_snps_inpeaks.drop_duplicates()
    print(control_snps_inpeaks.shape)

In [ ]:
#scoring mutations
for zz in peak_files[13:14]:
    name=re.sub('_$','',zz.split('ext')[0].split('chd_cases_')[1].replace('all_','all'))
    print(name)
    case_snps_in_peaks=pd.read_csv('mutations/chd_cases_'+str(name)+'_ext.bed',sep='\t',header=None)
    case_snps_in_peaks.columns=['Chrom','Pos0','Pos']
    case_snps_in_peaks.shape
    control_snps_in_peaks=pd.read_csv('mutations/chd_controls_'+str(name)+'_ext.bed',sep='\t',header=None)
    control_snps_in_peaks.columns=['Chrom','Pos0','Pos']
    snps_inpeaks=chd_pcgc.merge(case_snps_in_peaks,on=['Chrom','Pos0','Pos'])
    print(snps_inpeaks.shape)
    snps_inpeaks=snps_inpeaks.drop_duplicates()
    control_snps_inpeaks=controls.merge(control_snps_in_peaks,on=['Chrom','Pos0','Pos'])
    control_snps_inpeaks=control_snps_inpeaks.drop_duplicates()
    print(control_snps_inpeaks.shape)
    snps_inpeaks.to_csv("chd_final_0base_"+str(name)+".tsv",header=True,index=False,sep='\t')
    control_snps_inpeaks.to_csv("chd_control_final_0base_"+str(name)+".tsv",header=True,index=False,sep='\t')
    for zz1 in range(0,5):
        #load the model! 
        model_num=str(zz1)
        custom_objects={"recall":recall,
                    "sensitivity":recall,
                    "specificity":specificity,
                    "fpr":fpr,
                    "fnr":fnr,
                    "precision":precision,
                    "f1":f1,
                    "ambig_binary_crossentropy":ambig_binary_crossentropy,
                    "ambig_mean_absolute_error":ambig_mean_absolute_error,
                    "ambig_mean_squared_error":ambig_mean_squared_error,
                    "MultichannelMultinomialNLL":MultichannelMultinomialNLL}
        get_custom_objects().update(custom_objects)
        import json 
        model=model_from_json(open("weights/"+name+"."+str(model_num)+".arch",'r').read())
        model.load_weights("weights/"+name+"."+str(model_num)+".weights")
        #reference allele sequence generator 
        ref_gen=SNPGenerator(bed_path="chd_final_0base_"+name+".tsv",
                 chrom_col="Chrom",
                 pos_col="Pos0",
                 allele_col="Ref",
                 rsid_col='rsid',
                 flank_size=1057,
                 ref_fasta="GRCh38.fasta",
                 expand_dims=False)
        #alternate allele sequence generator 
        alt_gen=SNPGenerator(bed_path="chd_final_0base_"+name+".tsv",
                 chrom_col="Chrom",
                 pos_col="Pos0",
                 allele_col="Alt",
                 rsid_col='rsid',
                 flank_size=1057,
                 ref_fasta="GRCh38.fasta",
                 expand_dims=False)
        #get the reference allele predictions 
        count_preds={} 
        profile_preds={} 
        snp_to_seq={} 
        for i in range(0,len(ref_gen)):
            print(str(i))
            cur_x=ref_gen[i] 
            batch_rsids=cur_x[0] 
            batch_preds=model.predict(cur_x[1])
            batch_preds_profile=batch_preds[0]
            batch_preds_count=batch_preds[1] 
            for batch_index in range(len(batch_rsids)): 
                cur_rsid=batch_rsids[batch_index]
                snp_to_seq[cur_rsid]={} 
                snp_to_seq[cur_rsid]['ref']=cur_x[1][batch_index,:,:]
                cur_pred_profile=batch_preds_profile[batch_index,:,:]
                cur_pred_count=batch_preds_count[batch_index,:][0]
                count_preds[cur_rsid]={}
                count_preds[cur_rsid]['ref']=cur_pred_count
                profile_preds[cur_rsid]={}
                profile_preds[cur_rsid]['ref']=cur_pred_profile 
        #get the alternate allele predictions 
        for i in range(0,len(alt_gen)):
            print(str(i))
            cur_x=alt_gen[i] 
            batch_rsids=cur_x[0] 
            batch_preds=model.predict(cur_x[1])
            batch_preds_profile=batch_preds[0]
            batch_preds_count=batch_preds[1] 
            for batch_index in range(len(batch_rsids)): 
                cur_rsid=batch_rsids[batch_index]
                snp_to_seq[cur_rsid]['alt']=cur_x[1][batch_index,:,:]
                cur_pred_profile=batch_preds_profile[batch_index,:,:]
                cur_pred_count=batch_preds_count[batch_index,:][0]
                count_preds[cur_rsid]['alt']=cur_pred_count
                profile_preds[cur_rsid]['alt']=cur_pred_profile 
        count_preds_cases=count_preds
        profile_preds_cases=profile_preds
        snp_to_seq_cases=snp_to_seq
        #
        #sum and then log
        log_sumprob_ratios_100=[]
        log_sumprob_ratios_100[:]=[]
        for i in profile_preds_cases.keys():
            pred_prob_ref=softmax(profile_preds_cases[i]['ref'],axis=0)
            pred_prob_alt=softmax(profile_preds_cases[i]['alt'],axis=0)
            log_sumprob_ratios_100.append(np.log(np.sum(pred_prob_ref[450:550])/np.sum(pred_prob_alt[450:550])))
        #
        local_sumcount_changes_100=[]
        local_sumcount_changes_100[:]=[]
        for i in profile_preds_cases.keys():
            pred_prob_ref=softmax(profile_preds_cases[i]['ref'],axis=0)
            pred_prob_alt=softmax(profile_preds_cases[i]['alt'],axis=0)
            count_pred_ref=count_preds_cases[i]['ref']
            count_pred_alt=count_preds_cases[i]['alt']
            local_sumcount_changes_100.append(np.log( ((np.sum(np.exp(count_pred_ref)*pred_prob_ref[450:550])) / (np.sum(np.exp(count_pred_alt)*pred_prob_alt[450:550])))))
        #
        local_count_changes_100=[]
        local_count_changes_100[:]=[]
        for i in profile_preds_cases.keys():
            pred_prob_ref=softmax(profile_preds_cases[i]['ref'],axis=0)
            pred_prob_alt=softmax(profile_preds_cases[i]['alt'],axis=0)
            count_pred_ref=count_preds_cases[i]['ref']
            count_pred_alt=count_preds_cases[i]['alt']
            local_count_changes_100.append(np.sum(np.exp(count_pred_ref)*pred_prob_ref[450:550]-np.exp(count_pred_alt)*pred_prob_alt[450:550]))
        profile_preds_df=pd.DataFrame.from_dict(profile_preds_cases,orient='index')
        profile_preds_df['local_sumcount_changes_100']=local_sumcount_changes_100
        #sum and log 
        profile_preds_df['log_sumprob_ratios_100']=log_sumprob_ratios_100
        #reference allele sequence generator 
        ref_gen=SNPGenerator(bed_path='chd_control_final_0base_'+str(name)+'.tsv',
                 chrom_col="Chrom",
                 pos_col="Pos0",
                 allele_col="Ref",
                 rsid_col='rsid',
                 flank_size=1057,
                 ref_fasta="GRCh38.fasta",
                 expand_dims=False)
        #alternate allele sequence generator 
        alt_gen=SNPGenerator(bed_path='chd_control_final_0base_'+str(name)+'.tsv',
                 chrom_col="Chrom",
                 pos_col="Pos0",
                 allele_col="Alt",
                 rsid_col='rsid',
                 flank_size=1057,
                 ref_fasta="GRCh38.fasta",
                 expand_dims=False)
        #get the reference allele predictions 
        count_preds={} 
        profile_preds={} 
        snp_to_seq={} 
        for i in range(0,len(ref_gen)):
            print(str(i))
            cur_x=ref_gen[i] 
            batch_rsids=cur_x[0] 
            batch_preds=model.predict(cur_x[1])
            batch_preds_profile=batch_preds[0]
            batch_preds_count=batch_preds[1] 
            for batch_index in range(len(batch_rsids)): 
                cur_rsid=batch_rsids[batch_index]
                snp_to_seq[cur_rsid]={} 
                snp_to_seq[cur_rsid]['ref']=cur_x[1][batch_index,:,:]
                cur_pred_profile=batch_preds_profile[batch_index,:,:]
                cur_pred_count=batch_preds_count[batch_index,:][0]
                count_preds[cur_rsid]={}
                count_preds[cur_rsid]['ref']=cur_pred_count
                profile_preds[cur_rsid]={}
                profile_preds[cur_rsid]['ref']=cur_pred_profile 
        #get the alternate allele predictions 
        for i in range(0,len(alt_gen)):
            print(str(i))
            cur_x=alt_gen[i] 
            batch_rsids=cur_x[0] 
            batch_preds=model.predict(cur_x[1])
            batch_preds_profile=batch_preds[0]
            batch_preds_count=batch_preds[1] 
            for batch_index in range(len(batch_rsids)): 
                cur_rsid=batch_rsids[batch_index]
                snp_to_seq[cur_rsid]['alt']=cur_x[1][batch_index,:,:]
                cur_pred_profile=batch_preds_profile[batch_index,:,:]
                cur_pred_count=batch_preds_count[batch_index,:][0]
                count_preds[cur_rsid]['alt']=cur_pred_count
                profile_preds[cur_rsid]['alt']=cur_pred_profile 
        count_preds_controls=count_preds
        profile_preds_controls=profile_preds
        snp_to_seq_controls=snp_to_seq
        #sum and then log
        log_sumprob_ratios_100=[]
        #
        log_sumprob_ratios_100[:]=[]
        for i in profile_preds_controls.keys():
            pred_prob_ref=softmax(profile_preds_controls[i]['ref'],axis=0)
            pred_prob_alt=softmax(profile_preds_controls[i]['alt'],axis=0)
            log_sumprob_ratios_100.append(np.log(np.sum(pred_prob_ref[450:550])/np.sum(pred_prob_alt[450:550])))
        #
        local_sumcount_changes_100=[]
        local_sumcount_changes_100[:]=[]
        for i in profile_preds_controls.keys():
            pred_prob_ref=softmax(profile_preds_controls[i]['ref'],axis=0)
            pred_prob_alt=softmax(profile_preds_controls[i]['alt'],axis=0)
            count_pred_ref=count_preds_controls[i]['ref']
            count_pred_alt=count_preds_controls[i]['alt']
            local_sumcount_changes_100.append(np.log( ((np.sum(np.exp(count_pred_ref)*pred_prob_ref[450:550])) / (np.sum(np.exp(count_pred_alt)*pred_prob_alt[450:550])))))
        #
        profile_preds_df_controls=pd.DataFrame.from_dict(profile_preds_controls,orient='index')
        #sum and log counts
        profile_preds_df_controls['local_sumcount_changes_100']=local_sumcount_changes_100
        #sum and log 
        profile_preds_df_controls['log_sumprob_ratios_100']=log_sumprob_ratios_100
        dict_var_Score_cases[name+'_'+str(model_num)]=profile_preds_df
        dict_var_Score_controls[name+'_'+str(model_num)]=profile_preds_df_controls
    

In [ ]:
dict_var_Score_cases.keys()

In [ ]:
dict_var_Score_sumprob_cases_100_odds={}
dict_var_Score_sumprob_controls_100_odds={}
#
dict_var_Score_sumcounts_cases_100={}
dict_var_Score_sumcounts_controls_100={}

In [ ]:
import tensorflow_probability as tfp
peak_files=glob.glob('mutations/chd_cases_*_ext.bed')
for zz in peak_files[13:14]:
    name=re.sub('_$','',zz.split('ext')[0].split('chd_cases_')[1].replace('all_','all'))
    print(name)
    c0_fold0_cases=dict_var_Score_cases[name+'_0']
    c0_fold0_controls=dict_var_Score_controls[name+'_0']
    c0_fold1_cases=dict_var_Score_cases[name+'_1']
    c0_fold1_controls=dict_var_Score_controls[name+'_1']
    c0_fold2_cases=dict_var_Score_cases[name+'_2']
    c0_fold2_controls=dict_var_Score_controls[name+'_2']
    c0_fold3_cases=dict_var_Score_cases[name+'_3']
    c0_fold3_controls=dict_var_Score_controls[name+'_3']
    c0_fold4_cases=dict_var_Score_cases[name+'_4']
    c0_fold4_controls=dict_var_Score_controls[name+'_4']
    del c0_fold0_cases['ref']
    del c0_fold0_cases['alt']
    del c0_fold0_controls['ref']
    del c0_fold0_controls['alt']
    #
    del c0_fold1_cases['ref']
    del c0_fold1_cases['alt']
    del c0_fold1_controls['ref']
    del c0_fold1_controls['alt']
    #
    del c0_fold2_cases['ref']
    del c0_fold2_cases['alt']
    del c0_fold2_controls['ref']
    del c0_fold2_controls['alt']
    #
    del c0_fold3_cases['ref']
    del c0_fold3_cases['alt']
    del c0_fold3_controls['ref']
    del c0_fold3_controls['alt']
    #
    del c0_fold4_cases['ref']
    del c0_fold4_cases['alt']
    del c0_fold4_controls['ref']
    del c0_fold4_controls['alt']
    #
    c0_fold0_cases.columns=['local_sumcount_changes_100_1','log_sumprob_ratios_100_1']
    c0_fold1_cases.columns=['local_sumcount_changes_100_2','log_sumprob_ratios_100_2']
    c0_fold2_cases.columns=['local_sumcount_changes_100_3','log_sumprob_ratios_100_3']
    c0_fold3_cases.columns=['local_sumcount_changes_100_4','log_sumprob_ratios_100_4']
    c0_fold4_cases.columns=['local_sumcount_changes_100_5','log_sumprob_ratios_100_5']
    c0_fold0_controls.columns=['local_sumcount_changes_100_1','log_sumprob_ratios_100_1']
    c0_fold1_controls.columns=['local_sumcount_changes_100_2','log_sumprob_ratios_100_2']
    c0_fold2_controls.columns=['local_sumcount_changes_100_3','log_sumprob_ratios_100_3']
    c0_fold3_controls.columns=['local_sumcount_changes_100_4','log_sumprob_ratios_100_4']
    c0_fold4_controls.columns=['local_sumcount_changes_100_5','log_sumprob_ratios_100_5']
    #
    #log of sum counts
    #
    c0_local_sumcount_changes_100_cases=c0_fold0_cases[['local_sumcount_changes_100_1']].merge(c0_fold1_cases[['local_sumcount_changes_100_2']],left_index=True,right_index=True).\
    merge(c0_fold2_cases[['local_sumcount_changes_100_3']],left_index=True,right_index=True).\
    merge(c0_fold3_cases[['local_sumcount_changes_100_4']],left_index=True,right_index=True).\
    merge(c0_fold4_cases[['local_sumcount_changes_100_5']],left_index=True,right_index=True)
    #
    c0_local_sumcount_changes_100_controls=c0_fold0_controls[['local_sumcount_changes_100_1']].merge(c0_fold1_controls[['local_sumcount_changes_100_2']],left_index=True,right_index=True).\
    merge(c0_fold2_controls[['local_sumcount_changes_100_3']],left_index=True,right_index=True).\
    merge(c0_fold3_controls[['local_sumcount_changes_100_4']],left_index=True,right_index=True).\
    merge(c0_fold4_controls[['local_sumcount_changes_100_5']],left_index=True,right_index=True)
    ###############
    #log of sum
    #
    c0_local_sumprob_ratios_odd100_cases=c0_fold0_cases[['log_sumprob_ratios_100_1']].merge(c0_fold1_cases[['log_sumprob_ratios_100_2']],left_index=True,right_index=True).\
    merge(c0_fold2_cases[['log_sumprob_ratios_100_3']],left_index=True,right_index=True).\
    merge(c0_fold3_cases[['log_sumprob_ratios_100_4']],left_index=True,right_index=True).\
    merge(c0_fold4_cases[['log_sumprob_ratios_100_5']],left_index=True,right_index=True)
    #
    c0_local_sumprob_ratios_odd100_controls=c0_fold0_controls[['log_sumprob_ratios_100_1']].merge(c0_fold1_controls[['log_sumprob_ratios_100_2']],left_index=True,right_index=True).\
    merge(c0_fold2_controls[['log_sumprob_ratios_100_3']],left_index=True,right_index=True).\
    merge(c0_fold3_controls[['log_sumprob_ratios_100_4']],left_index=True,right_index=True).\
    merge(c0_fold4_controls[['log_sumprob_ratios_100_5']],left_index=True,right_index=True)
    #
    ######sumcounts
    c0_local_sumcount_changes_100_cases['mean']=np.mean(c0_local_sumcount_changes_100_cases,axis=1)
    c0_local_sumcount_changes_100_controls['mean']=np.mean(c0_local_sumcount_changes_100_controls,axis=1)
    c0_local_sumcount_changes_200_cases['mean']=np.mean(c0_local_sumcount_changes_200_cases,axis=1)
    c0_local_sumcount_changes_200_controls['mean']=np.mean(c0_local_sumcount_changes_200_controls,axis=1)
    c0_local_sumcount_changes_300_cases['mean']=np.mean(c0_local_sumcount_changes_300_cases,axis=1)
    c0_local_sumcount_changes_300_controls['mean']=np.mean(c0_local_sumcount_changes_300_controls,axis=1)
    ######sumprob
    c0_local_sumprob_ratios_odd100_cases['mean']=np.mean(c0_local_sumprob_ratios_odd100_cases,axis=1)
    c0_local_sumprob_ratios_odd100_controls['mean']=np.mean(c0_local_sumprob_ratios_odd100_controls,axis=1)
    c0_local_sumprob_ratios_odd200_cases['mean']=np.mean(c0_local_sumprob_ratios_odd200_cases,axis=1)
    c0_local_sumprob_ratios_odd200_controls['mean']=np.mean(c0_local_sumprob_ratios_odd200_controls,axis=1)
    c0_local_sumprob_ratios_odd300_cases['mean']=np.mean(c0_local_sumprob_ratios_odd300_cases,axis=1)
    c0_local_sumprob_ratios_odd300_controls['mean']=np.mean(c0_local_sumprob_ratios_odd300_controls,axis=1)
    #####
    dict_var_Score_sumprob_cases_100_odds[name+'_cases_logsumprob']=c0_local_sumprob_ratios_odd100_cases
    dict_var_Score_sumprob_controls_100_odds[name+'_controls_logsumprob']=c0_local_sumprob_ratios_odd100_controls
    #
    dict_var_Score_sumcounts_cases_100[name+'_cases_logsumcounts']=c0_local_sumcount_changes_100_cases
    dict_var_Score_sumcounts_controls_100[name+'_cases_logsumcounts']=c0_local_sumcount_changes_100_controls



    
    
    
    